In [181]:
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.cross_decomposition import PLSCanonical
import random
import pandas as pd
from sys import exit
from sklearn import metrics
import matplotlib.pyplot as plt
import csv

# USUAL PLS

def perform_pls(X, Y, n_comp, mean):
    PLS_model = PLSRegression(n_comp, scale=True) #initialize a generic PLS model with parameters
    
    PLS_model.fit(X, Y) #ACTUALLY doing the PLS, fit model to data
    
    pred = PLS_model.predict(X) #predict the values on the train set
    
    Y_temp = np.array(Y)
    pred_temp = pred.flatten()
    
    diff = Y_temp - pred_temp  
    
    #print "Y_temp before: ", Y_temp
    #print "pred_temp before: ", pred_temp
    
    Y_temp = Y_temp+mean
    pred_temp = pred_temp+mean
    for j in range(len(pred_temp)):
        if pred_temp[j]>1:
            pred_temp[j]=1 
        elif pred_temp[j]<0:
            pred_temp[j]=0
    pred_temp = np.rint(pred_temp)
    for j in range(len(Y_temp)):
        if Y_temp[j]>1:
            Y_temp[j]=1 
        elif Y_temp[j]<0:
             Y_temp[j]=0
    pred_temp = np.rint(pred_temp)
    
    #print "Y_temp after: ", Y_temp
    #print "pred_temp after: ", pred_temp
    
    #print "Training ROC curve:"
    fpr, tpr, auc = get_roc_auc(Y_temp, pred_temp)
    #plot_roc_curve(fpr, tpr, auc)
    
    #diff = Y_temp - pred_temp             #see the difference
    #print "Difference: ",diff
    #print "Type of difference: ", type(diff)
    train_error = sum(diff*diff) #calculate square error
    #print "Train AUC: ", auc
    #print "Training error: ", train_error
    return PLS_model, train_error, auc

In [182]:
def center_data(matrix, mean=None):
    if mean is None:
        matrix = matrix - np.mean(matrix, axis=0)
    else:
        matrix = matrix - mean
    return matrix

In [183]:
def scale_data(matrix, deviation, ddof):
    if deviation is None:
        matrix = matrix/np.std(matrix, axis=0, ddof=ddof)
    else:
        matrix = matrix/deviation
    return matrix

In [184]:
def get_roc_auc(labels, predictions):
    fpr, tpr, thresholds = metrics.roc_curve(labels, predictions) #compute precision-recall curve
    if np.isnan(tpr).any():
        tpr = np.ones(tpr.shape)
    if np.isnan(fpr).any():
        fpr = np.ones(fpr.shape)
    auc = metrics.auc(fpr, tpr) #compute area under the curve for this run
    return fpr, tpr, auc

In [185]:
def plot_roc_curve(fpr, tpr, auc):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [186]:
#MULTILEVEL PLS
def perform_multilevel_pls(X, Y, subj, unique_subj, num_unique_subj, num_subj, scaling, par, n_comp, ddof):
    #mean-centering
    X_centered = center_data(X) #mean centering of data
    Y_centered = center_data(Y) #FIXME :think about Y and scaling/centering

    #split matrix into between and within subject variations
    Xb, Xw = split_between_within_subject_matrix(X_centered, subj, unique_subj, num_unique_subj, num_subj)
    
    #scaling (if necessary and specified)
    if scaling: #scale the data, if the flag is true
        X_scaled = scale_data(X_centered,None, ddof)
        Y_scaled = scale_data(Y_centered, None, ddof)                          
        #Y_scaled = Y_centered
        Xb_scaled = scale_data(Xb,None, ddof)
        Xw_scaled = scale_data(Xw,None, ddof)
    else:
        X_scaled = X_centered
        Y_scaled = Y_centered
        Xb_scaled = Xb
        Xw_scaled = Xw
    
    #which matrix are we interested in
    #print "Using (in the usual PLS) the matrix ", par
    if par=="all":
        model, train_error, train_auc = perform_pls(X_scaled, Y_scaled, n_comp=n_comp, mean=np.mean(Y)) #perform usual pls on whole dataset
    elif par=="between":
        model, train_error, train_auc = perform_pls(X=Xb_scaled, Y=Y_scaled, n_comp=n_comp, mean=np.mean(Y)) #perform usual PLS on between subject variation
    elif par=="within":
        model, train_error, train_auc = perform_pls(Xw_scaled, Y_scaled, n_comp=7, mean=np.mean(Y)) #perform usual PLS on within subject variation
    
    return model, train_error, train_auc, Xb_scaled, Xw_scaled

In [187]:
def separate_train_test(X, Y, subj, unique_subj, num_subj, num_unique_subj, num_subj_in_fold):
    
    X_train = None #initialization
    X_test = None
    Y_train = []
    Y_test = []
    subj_train = []
    subj_test = []
    
    test_subj = np.random.choice(unique_subj, num_subj_in_fold, replace=False) #choose subjects for test set
    train_subj = np.setdiff1d(unique_subj, test_subj)                           #other subjects => train set

    #create the test and train dataset from matrix X with chosen subjects
    while(True):
        for j in range(num_subj):
            if subj[j] in train_subj: 
                if X_train is not None:
                    X_train = np.vstack([X_train, X[j]]) 
                else : 
                    X_train = np.matrix(X[j])
                Y_train.append(Y[j]) 
                subj_train.append(subj[j])
            else:
                if X_test is not None:
                    X_test = np.vstack([X_test, X[j]])
                else : 
                    X_test = np.matrix(X[j]) 
                Y_test.append(Y[j]) 
                subj_test.append(subj[j])
        n_unique_test = len(np.unique(Y_train))
        if n_unique_test>1:
            break
        
        #X_train = X_train.astype(int)                   #FIXME scaling
        #X_test = X_test.astype(int)
                
    num_train_subj = len(subj_train)          #how many entries in train dataset
    num_test_subj= num_subj - num_train_subj  #how many entries in test dataset
    num_unique_train_subj = num_unique_subj - num_subj_in_fold
    num_unique_test_subj = num_subj_in_fold
        
    return X_train, X_test, Y_train, Y_test, subj_train, subj_test, num_train_subj, num_test_subj, num_unique_train_subj, num_unique_test_subj, train_subj, test_subj

In [188]:
def separate_train_test_old(X, Y, subj, unique_subj, num_subj, num_unique_subj, num_subj_in_fold):
    
    X_train = None #initialization
    X_test = None
    Y_train = []
    Y_test = []
    subj_train = []
    subj_test = []
    
    test_subj = np.random.choice(unique_subj, num_subj_in_fold, replace=False) #choose subjects for test set
    train_subj = np.setdiff1d(unique_subj, test_subj)                           #other subjects => train set

    #create the test and train dataset from matrix X with chosen subjects
    for j in range(num_subj):
        if subj[j] in train_subj: 
            if X_train is not None:
                X_train = np.vstack([X_train, X[j]]) 
            else : 
                X_train = np.matrix(X[j])
            Y_train.append(Y[j]) 
            subj_train.append(subj[j])
        else:
            if X_test is not None:
                X_test = np.vstack([X_test, X[j]])
            else : 
                X_test = np.matrix(X[j]) 
            Y_test.append(Y[j]) 
            subj_test.append(subj[j])

        #X_train = X_train.astype(int)                   #FIXME scaling
        #X_test = X_test.astype(int)
                
    num_train_subj = len(subj_train)          #how many entries in train dataset
    num_test_subj= num_subj - num_train_subj  #how many entries in test dataset
    num_unique_train_subj = num_unique_subj - num_subj_in_fold
    num_unique_test_subj = num_subj_in_fold
        
    return X_train, X_test, Y_train, Y_test, subj_train, subj_test, num_train_subj, num_test_subj, num_unique_train_subj, num_unique_test_subj, train_subj, test_subj

In [189]:
#CROSS-VALIDATION
def cross_validation(X, Y, subj, unique_subj, num_subj, num_unique_subj, num_folds, num_repeats, scaling, par, n_comp, ddof):
    
    error = 0 #initialization
    auc = 0
    
    #calculate number of subjects per 1 fold in cross-validation
    if num_unique_subj>num_folds: 
        num_subj_in_fold = num_unique_subj/num_folds
    else:
        num_subj_in_fold = num_unique_subj

    for i in range(num_repeats): #repeat cross_validation as many times as specified
        X_train, X_test, Y_train, Y_test, subj_train, subj_test, num_train_subj, num_test_subj, num_unique_train_subj, num_unique_test_subj, train_subj, test_subj = separate_train_test_old(X, Y, subj, unique_subj, num_subj, num_unique_subj, num_subj_in_fold)
        
        #perform multilevel pls on TRAIN dataset
        model, train_error, train_auc, Xb_train, Xw_train = perform_multilevel_pls(X=X_train, Y=Y_train, subj=subj_train, unique_subj=train_subj, num_unique_subj=num_unique_train_subj, num_subj=num_train_subj, scaling=scaling, par=par, n_comp=n_comp, ddof=ddof)
        
        #Xb_train = Xb_train.astype(int)
        #Xw_train = Xw_train.astype(int) #FIXME scaling
        
        X_train_mean = np.mean(X_train)
        Y_train_mean = np.mean(Y_train)
        
        X_centered_test = center_data(X_test, X_train_mean) #mean centering of data
        Y_centered_test = center_data(Y_test, Y_train_mean)                        #FIXME : centering and scaling
        #Y_centered_test = Y_test - np.mean(Y_train, axis=0)
        #X_centered_test =  X_centered_test.astype(int) #for std
        #Y_centered_test =  Y_centered_test.astype(int)
        

        #split test data into between and within subject variation
        Xb_test, Xw_test = split_between_within_subject_matrix(X=X_centered_test, subj=subj_test, unique_subj=test_subj,num_unique_subj=num_unique_test_subj, num_subj=num_test_subj )
        #Xb_test = Xb_test.astype(int)
        #Xw_test = Xw_test.astype(int) #FIXME scaling     

        if scaling: #scale the data, if the flag is true
            X_train_deviation = np.std(X_train, axis = 0, ddof=ddof)
            Y_train_deviation = np.std(Y_train, axis = 0, ddof=ddof)
            Xb_train_deviation = np.std(Xb_train, axis = 0, ddof=ddof)
            Xw_train_deviation = np.std(Xw_train, axis = 0, ddof=ddof)
            
            X_scaled_test = scale_data(X_centered_test, X_train_deviation, ddof)
            Y_scaled_test = scale_data(Y_centered_test, Y_train_deviation, ddof)
            
            Xb_scaled_test = scale_data(Xb_test, Xb_train_deviation, ddof)
            Xw_scaled_test = scale_data(Xw_test, Xw_train_deviation, ddof)
        else:
            
            X_scaled_test = X_centered_test
            Y_scaled_test = Y_centered_test
            Xb_scaled_test = Xb_test
            Xw_scaled_test = Xw_test
            
        #print "Use the model to predict the test values for matrix ", par   
        pred = model.predict(X_scaled_test) #predict test data with model trained on the training set
        #if only between or only within subject variations chosen, predict on that part of the matrix
        if par=="between":
             pred = model.predict(Xb_scaled_test)
        elif par=="within":
             pred = model.predict(Xw_scaled_test)   
            
        diff = Y_scaled_test - pred         #see the differences
        error = error + sum(sum(diff*diff)) #compute square error
        
        #bring labels and predictions to binary (0 or 1)
        Y_test = np.array(Y_test)
        pred = pred.flatten()+Y_train_mean
        for j in range(len(pred)):
            if pred[j]>1:
                pred[j]=1 
            elif pred[j]<0:
                pred[j]=0
        pred = np.rint(pred)
        
        #compute ROC curve and AUC
        fpr, tpr, auc_temp = get_roc_auc(Y_test, pred)
        auc = auc+auc_temp #sum of all AUC scores (to then get a mean AUC score)
        
        #plot ROC curve, if want
        #if i%10==0:
        #    plot_roc_curve(fpr, tpr, auc_temp)
        
    error = error/num_repeats #mean error for cross-validation
    auc = auc/num_repeats     #mean AUC score for cross validation
    
    print "Y: ", Y
    print "Y transposed: ", Y.transpose()
    print "Result of the multiplicaton: ", np.matmul(Y, Y.transpose())
    print "Number of subjects: ", num_subj
    print "Error: ", error
    print "To subtract:", error/(np.matmul(Y, Y.transpose())/num_subj)
    Q = 1 - error/(np.matmul(Y, Y.transpose())/num_subj)
    print "Mean cross-validation error: ", error
    print "Mean AUC: ", auc
    print "Q: ", Q
    return error, auc, Q

In [190]:
# FULL SCRIPT
def full_script(num_folds, num_repeats, scaling, num_permutations, par, filename, n_comp, ddof):
    #READ OR MAKE UP DATA
    X, Y, subj = read_data(filename)

    unique_subj = np.unique(subj) #create list of unique subjects
    num_subj = len(subj)        #the number of entries = number of subjects corresponding to entries (1 subject per entry)
    num_unique_subj = len(unique_subj) #the number of unique subjects
    error = 0 #initialization

    #PERFORMING MULTILEVEL PLS ON WHOLE DATASET
    full_model, full_train_error, full_train_auc, Xb, Xw = perform_multilevel_pls(X=X, Y=Y, subj=subj, unique_subj=unique_subj, num_subj=num_subj, num_unique_subj=num_unique_subj, scaling=scaling, par=par, n_comp=n_comp, ddof=ddof)
    
    #CROSS-VALIDATION
    crossval_error, crossval_auc, crossval_Q = cross_validation(X, Y, subj, unique_subj, num_subj, num_unique_subj, num_folds, num_repeats, scaling, par, n_comp=n_comp, ddof=ddof)
    
    #PERMUTATE
    print "Permutation errors: "
    permutation_error, permutation_auc, permutation_Q= validate_permutation(X=X, Y=Y, subj=subj, unique_subj=unique_subj, num_subj=num_subj, num_unique_subj=num_unique_subj, num_folds=num_folds, num_repeats=num_repeats, scaling=scaling, num_permutations=num_permutations, par=par, n_comp=n_comp, ddof=ddof)
    
    return 0

In [191]:
#split data matrix nito between and within subject variations
def split_between_within_subject_matrix(X, subj, unique_subj, num_unique_subj, num_subj): #SPLIT MATRIX 
    Xb = np.zeros(X.shape) #initialization
    Xw = np.zeros(X.shape)
    means = np.zeros((num_unique_subj, X.shape[1]))
    for j in range(num_unique_subj): #go through all unique subjects
        idx = np.where(np.array(subj)==unique_subj[j]) #find indexes of all entries for a certain subject (unique_subj[j])
        means[j] = np.mean(X[idx[0]], axis=0)   #calculate mean for each subject unique_subj[j]        
    for i in range(num_subj): #go through subjects of all entries
        k = np.where(unique_subj==subj[i]) #find the index of the subject corresponding to subj[i] in unique_subj
        Xb[i] = means[k[0][0]]                  #create a matrix where all entries for subject = mean (between subject variation)
    
    Xw = X - Xb             #FIXME make sure of within subject variation
    return Xb, Xw

In [192]:
#PERMUTATION
def validate_permutation(X, Y, subj, unique_subj, num_subj, num_unique_subj, num_folds, num_repeats, scaling, num_permutations, par, n_comp, ddof):
    err = [] #initialization
    auc = []
    Q = []
    for i in range(num_permutations):#perform permutations as many times as specified
        Y_temp = Y  #create temporary labels which then shuffle/permutate, so original is left untouched
        
        np.random.shuffle(Y_temp) #randomly shuffle the Y(labels) vector. Checks if your results will be similar
        
        #perform cross-validation for each permutation and get an array of accuracies when permutated
        err_temp, auc_temp, Q_temp = cross_validation(X, Y_temp, subj, unique_subj, num_subj, num_unique_subj, num_folds, num_repeats, scaling, par, n_comp, ddof)
        err.append(err_temp)
        auc.append(auc_temp)
        Q.append(Q_temp)
    if len(err) >0 :
        print "Mean permutated squared error : ", np.mean(err)
        print "Mean permutated auc : ", np.mean(auc)
        print "Mean permutated Q: ", np.mean(Q)
    else:
        print "No permutations performed"
    return err, auc, Q

In [193]:
def read_data(file_name=None):
    if file_name is None: #if no input file specified, make up data
        print "WARNING : results are nonsence, working on MADE UP data. Only for debugging, testing purposes."
        X =  np.random.rand(140, 300) 
        Y =  np.random.rand(140, 1)
        subjects = [random.randint(1, 35) for i in range(140)]
 
    else:
        data = pd.read_excel(file_name) #read the excel file
        subjects = data.head().ix[-2].values[0:148].astype(int) #take relevant values as subkect id's, convert to int
        Y = (data.head().ix[-3].values[0:148]=='risk').astype(int)#take relevant values as labels, convert to int
        X = data.as_matrix()[7:, :-9].transpose().astype(int)   #take the matabolite matrix, transpose, convert to int
    return X, Y, subjects

In [194]:
full_script(num_folds=5, num_repeats=200, scaling=True, num_permutations=0, par='between', filename='OGTT_INPUT.xlsx', n_comp=8, ddof=1)

/home/anna/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
/home/anna/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
/home/anna/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


Y:  [0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1
 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1
 0 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Y transposed:  [0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1
 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1
 0 1 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0]
Result of the multiplicaton:  63
Number of subjects:  148
Error:  3.402936940239801e+17
To subtract: inf
Mean cross-validation error:  3.402936940239801e+17
Mean AUC:  0.5106198509504134
Q:  -inf
Permutation errors: 
No permutations performed


/home/anna/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: divide by zero encountered in double_scalars
/home/anna/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in double_scalars


0